# TODO:

1. accuracy CANNOT be 100% (even for SVM??)
1. study performance of models
1. remove bad features (e.g. id)
1. add remaining good features
1. use pandas.get_dummies or scikit OneHotEncoder for one-hot encoding (start with pandas)
1. model and train regression models predicting actual return rate
1. thinks how to continuously train models (every month, or something like that), productionize?



In [35]:
import time

import definitions
import pandas_helper
import scikit_helper
import lendingclub_helper

import pandas as pd
import numpy as np


In [2]:
dataFrames = {}
for key in definitions.dataFiles:
    print("Loading data for " + key)
    dataFrames[key] = pandas_helper.readData(definitions.dataFiles[key])

Loading data for 2007-2011
Loading data for 2012-2013
Loading data for 2014
Loading data for 2015
Loading data for 2016Q1
Loading data for 2016Q2
Loading data for 2016Q3
Loading data for 2016Q4
Loading data for 2017Q1
Loading data for 2017Q2
Loading data for 2017Q3
Loading data for 2017Q4


In [3]:
for key in dataFrames.keys():
    print("Building features for " + key)
    lendingclub_helper.buildFeatures(dataFrames[key])
    featureColumns = lendingclub_helper.getFeatureColumns(dataFrames[key])
    assert(not pandas_helper.columnsHaveNull(dataFrames[key], featureColumns.tolist()))

Building features for 2007-2011
Building features for 2012-2013
Building features for 2014
Building features for 2015
Building features for 2016Q1
Building features for 2016Q2
Building features for 2016Q3
Building features for 2016Q4
Building features for 2017Q1
Building features for 2017Q2
Building features for 2017Q3
Building features for 2017Q4


In [4]:
for key in dataFrames.keys():
    print("Building labels for " + key)
    lendingclub_helper.buildLabels(dataFrames[key])
    print(dataFrames[key].shape)

Building labels for 2007-2011
(42535, 1192)
Building labels for 2012-2013
(188181, 1193)
Building labels for 2014
(235629, 1219)
Building labels for 2015
(421095, 1269)
Building labels for 2016Q1
(133887, 1244)
Building labels for 2016Q2
(97854, 1228)
Building labels for 2016Q3
(99120, 1228)
Building labels for 2016Q4
(103546, 1222)
Building labels for 2017Q1
(96779, 1216)
Building labels for 2017Q2
(105451, 1206)
Building labels for 2017Q3
(122701, 1211)
Building labels for 2017Q4
(118648, 1239)


In [41]:
col = "hardship_type"
all_values = []
for key in dataFrames.keys():
    print(key)
#     print(col in dataFrames[key].columns)
#     print(dataFrames[key][col].describe())
#     print(dataFrames[key][col].unique())
    print(dataFrames[key][col].unique().shape)
    all_values += list(dataFrames[key][col].unique())

print(set(all_values))


2007-2011
(1,)
2012-2013
(2,)
2014
(2,)
2015
(2,)
2016Q1
(2,)
2016Q2
(2,)
2016Q3
(2,)
2016Q4
(2,)
2017Q1
(2,)
2017Q2
(2,)
2017Q3
(2,)
2017Q4
(2,)
{nan, 'INTEREST ONLY-3 MONTHS DEFERRAL'}


In [20]:
dataFrames["2015"]["sec_app_earliest_cr_line"].describe()

count     0
unique    0
Name: sec_app_earliest_cr_line, dtype: int64

In [48]:
col = "f_hardship_type"
for key in dataFrames.keys():
    print(key)
#     lendingclub_helper.buildOneHotEncodingFeature(dataFrames[key], "hardship_type")
    cols = [c for c in dataFrames[key].columns if col in c]
    print(dataFrames[key][cols].head())
print(cols)

2007-2011
Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4]
2012-2013
   f_hardship_type_INTEREST ONLY-3 MONTHS DEFERRAL
0                                                0
1                                                0
2                                                0
3                                                0
4                                                0
2014
   f_hardship_type_INTEREST ONLY-3 MONTHS DEFERRAL
0                                                0
1                                                0
2                                                0
3                                                0
4                                                0
2015
   f_hardship_type_INTEREST ONLY-3 MONTHS DEFERRAL
0                                                0
1                                                0
2                                                0
3                                                0
4                                                0
2

In [51]:
df = dataFrames[key]
[c for c in df.columns if 'f_hardship_type' in c]

# for key in dataFrames.keys():
#     print(key)
#     print(lendingclub_helper.getFeatureColumns(dataFrames[key]).shape)


['f_hardship_type_INTEREST ONLY-3 MONTHS DEFERRAL']

In [5]:
for key in dataFrames.keys():
    print("Getting finished loans for " + key)
    labelColumns = lendingclub_helper.getLabelColumns(dataFrames[key])
    dataFrames[key] = lendingclub_helper.getFinishedLoans(dataFrames[key])
    assert(not pandas_helper.columnsHaveNull(dataFrames[key], labelColumns.tolist()))
    print(dataFrames[key].shape)


Getting finished loans for 2007-2011
(42535, 236)
Getting finished loans for 2012-2013
(181326, 236)
Getting finished loans for 2014
(210765, 236)
Getting finished loans for 2015
(245635, 236)
Getting finished loans for 2016Q1
(57531, 236)
Getting finished loans for 2016Q2
(36764, 236)
Getting finished loans for 2016Q3
(34214, 236)
Getting finished loans for 2016Q4
(28809, 236)
Getting finished loans for 2017Q1
(19286, 236)
Getting finished loans for 2017Q2
(13916, 236)
Getting finished loans for 2017Q3
(8986, 236)
Getting finished loans for 2017Q4
(3871, 236)


In [18]:
key1 = "2012-2013"
key2 = "2017Q4"
filename = "../models/svc_{0}.pkl".format(key1)
clf = scikit_helper.loadModel(filename)
df = dataFrames[key2]
features = lendingclub_helper.getFeatureColumns(df)
labels = lendingclub_helper.getLabelColumns(df)
print(getModelAccuracy(clf, df, features, labels))

28
0.9927667269439421


In [11]:
def getModelAccuracy(clf, df, features, labels):
    X = df[features].values
    y = df[labels[0]].values
    y_p = clf.predict(X)
    print(sum(y != y_p))

    total = sum(y == y_p)
    assert(y.shape == y_p.shape)
    return total/len(y)


### Explanation
Rows: what model was trained on

Columns: which result was used to test

Accurary rounded to four decimal places

||2007-2011|2012-2013|2014|2015|2016Q1|2016Q2|2016Q3|2016Q4|2017Q1|2017Q2|2017Q3|2017Q4|
|----|----|----|----|----|----|----|----|----|----|----|----|----|----|
|**2007-2011**|1.0|0.8384|0.8152|0.7474|0.7271|0.7252|0.7464|0.7947|0.8321|0.8594|0.9289|0.9928|
|**2012-2013**|0.8488|1.0|0.8152|0.7474|0.7271|0.7252|0.7464|0.7947|0.8321|0.8594|0.9289|0.9928|
|**2014**|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|
|**2015**|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|
|**2016Q1**|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|
|**2016Q2**|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|
|**2016Q3**|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|
|**2016Q4**|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|
|**2017Q1**|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|
|**2017Q2**|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|
|**2017Q3**|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|
|**2017Q4**|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|sooka|


In [29]:
df = pandas_helper.readData("sample.csv")

In [36]:
# lendingclub_helper.buildDateFeature(df, "next_pymnt_d")
# lendingclub_helper.buildOneHotEncodingFeature(df, "hardship_type")
df["temp"] = pd.to_datetime(df["next_pymnt_d"]).dt.strftime('%b')

In [31]:
# l = [c for c in df.columns if "term" in c]
l = lendingclub_helper.getFeatureColumns(df)
l

Index(['f_hardship_type_INTEREST ONLY-3 MONTHS DEFERRAL', 'f_hardship_type_hardship_type_NULL'], dtype='object')

In [38]:
# df["next_pymnt_d"]
df["temp"].fillna("NULL")


0      Mar
1      NaT
2      Mar
3      Mar
4      Mar
5      Mar
6      Mar
7      NaT
8      NaT
9      Mar
10     NaT
11     Mar
12     Mar
13     NaT
14     NaT
15     Mar
16     NaT
17     NaT
18     NaT
19     NaT
20     Mar
21     NaT
22     Mar
23     NaT
24     Mar
25     NaT
26     NaT
27     NaT
28     Mar
29     Mar
      ... 
970    Mar
971    NaT
972    NaT
973    NaT
974    Mar
975    NaT
976    NaT
977    Mar
978    NaT
979    NaT
980    NaT
981    Mar
982    Mar
983    NaT
984    Mar
985    Mar
986    NaT
987    Mar
988    NaT
989    NaT
990    NaT
991    Mar
992    Mar
993    Mar
994    NaT
995    Mar
996    Mar
997    NaT
998    Mar
999    Mar
Name: temp, Length: 1000, dtype: object

In [83]:
pd.to_datetime(df["next_pymnt_d"]).dt.strftime('%b')

0      NaT
1      NaT
2      Mar
3      Mar
4      Mar
5      NaT
6      NaT
7      Mar
8      NaT
9      Mar
10     NaT
11     NaT
12     Mar
13     Mar
14     Mar
15     Mar
16     May
17     Mar
18     NaT
19     NaT
20     NaT
21     Mar
22     Mar
23     NaT
24     NaT
25     Mar
26     NaT
27     Mar
28     NaT
29     NaT
      ... 
970    Mar
971    Mar
972    NaT
973    NaT
974    Mar
975    NaT
976    NaT
977    NaT
978    NaT
979    NaT
980    Mar
981    Mar
982    NaT
983    NaT
984    NaT
985    NaT
986    NaT
987    Mar
988    NaT
989    NaT
990    NaT
991    NaT
992    NaT
993    NaT
994    NaT
995    Mar
996    Mar
997    NaT
998    Mar
999    Mar
Name: next_pymnt_d, Length: 1000, dtype: object

In [70]:
now
np.timedelta64(1, "D")

numpy.timedelta64(1,'D')

In [78]:
# pd.to_datetime(df["next_pymnt_d"])
# now - pd.to_datetime(df["next_pymnt_d"])
df["test"] = (now - pd.to_datetime(df["next_pymnt_d"]))/np.timedelta64(1, "D")
df["test"].fillna(df["test"].mean())


0        56.190513
1        56.190513
2        44.970905
3        44.970905
4        44.970905
5        56.190513
6        56.190513
7        44.970905
8        56.190513
9        44.970905
10       56.190513
11       56.190513
12       44.970905
13       44.970905
14       44.970905
15       44.970905
16     2905.970905
17       44.970905
18       56.190513
19       56.190513
20       56.190513
21       44.970905
22       44.970905
23       56.190513
24       56.190513
25       44.970905
26       56.190513
27       44.970905
28       56.190513
29       56.190513
          ...     
970      44.970905
971      44.970905
972      56.190513
973      56.190513
974      44.970905
975      56.190513
976      56.190513
977      56.190513
978      56.190513
979      56.190513
980      44.970905
981      44.970905
982      56.190513
983      56.190513
984      56.190513
985      56.190513
986      56.190513
987      44.970905
988      56.190513
989      56.190513
990      56.190513
991      56.